In [2]:
import pandas as pd

# Load CSV files
events_df = pd.read_csv("events.csv")
results_df = pd.read_csv("results.csv")

# Inspect the data
print("Events DataFrame Info:")
print(events_df.info())
print("\nResults DataFrame Info:")
print(results_df.info())



Events DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Local Time  139 non-null    object
 1   Sex         139 non-null    object
 2   Event       139 non-null    object
 3   Round       139 non-null    object
dtypes: object(4)
memory usage: 4.5+ KB
None

Results DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3550 entries, 0 to 3549
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Local Time  3550 non-null   object 
 1   Sex         3550 non-null   object 
 2   Event       3550 non-null   object 
 3   Round       3550 non-null   object 
 4   Position    3401 non-null   float64
 5   Bib Number  3404 non-null   float64
 6   Name        3550 non-null   object 
 7   Country     3550 non-null   object 
dtypes: float64(2), object(6)
memory usage: 222

In [6]:
# Handle missing values
results_df["Bib Number"] = results_df["Bib Number"].fillna(0)
results_df["Position"] = results_df["Position"].fillna("N/A")

# Inspect the data
print("Events DataFrame Info:")
print(events_df.info())
print("\nResults DataFrame Info:")
print(results_df.info())


Events DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Local Time  139 non-null    object
 1   Sex         139 non-null    object
 2   Event       139 non-null    object
 3   Round       139 non-null    object
dtypes: object(4)
memory usage: 4.5+ KB
None

Results DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3550 entries, 0 to 3549
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Local Time  3550 non-null   object 
 1   Sex         3550 non-null   object 
 2   Event       3550 non-null   object 
 3   Round       3550 non-null   object 
 4   Position    3550 non-null   object 
 5   Bib Number  3550 non-null   float64
 6   Name        3550 non-null   object 
 7   Country     3550 non-null   object 
dtypes: float64(1), object(7)
memory usage: 222

In [8]:
# Fix inconsistent formatting
# Correct "Unpecified" to "Unspecified" in events.csv
events_df["Sex"] = events_df["Sex"].replace("Unpecified", "Unspecified")

# Remove commas in Event names (e.g., "10,000 Metres" -> "10000 Metres")
events_df["Event"] = events_df["Event"].str.replace(",", "")

events_df

,Local Time,Sex,Event,Round
0,08:00,Men,20 Kilometres Race Walk,Final
1,09:50,Women,20 Kilometres Race Walk,Final
2,10:05,Men,100 Metres,Decathlon
3,10:10,Men,Hammer Throw,Qualification Group A
4,10:15,Women,High Jump,Qualification
...,...,...,...,...
134,19:50,Men,5000 Metres,Final
135,20:15,Women,1500 Metres,Final
136,21:00,Men,4x400 Metres Relay,Final
137,21:14,Women,4x400 Metres Relay,Final


In [14]:
# Remove duplicates
events_df = events_df.drop_duplicates()
results_df = results_df.drop_duplicates()

In [15]:
# Handle special characters and extra spaces
# Clean up string columns (e.g., Name, Country, Event)
for col in ["Event", "Round", "Sex"]:
    events_df[col] = events_df[col].str.strip()
    results_df[col] = results_df[col].str.strip()

# Clean up Name and Country in results_df
results_df["Name"] = results_df["Name"].str.strip().str.replace(r"[^\w\s]", "", regex=True)
results_df["Country"] = results_df["Country"].str.strip()

In [16]:
# Validate and fix inconsistencies
# Ensure consistent capitalization for categorical columns
events_df["Sex"] = events_df["Sex"].str.title()
events_df["Event"] = events_df["Event"].str.title()
events_df["Round"] = events_df["Round"].str.title()

results_df["Sex"] = results_df["Sex"].str.title()
results_df["Event"] = results_df["Event"].str.title()
results_df["Round"] = results_df["Round"].str.title()

In [17]:
# Standardize data types
# Convert Local Time to proper time format (assuming HH:MM format) with debugging
def parse_time(time_str):
    # Handle NaN or non-string values
    if pd.isna(time_str) or not isinstance(time_str, str):
        print(f"Invalid time value (NaN or non-string): {time_str}")
        return None
    # Remove extra spaces and try parsing
    time_str = time_str.strip()
    try:
        return datetime.strptime(time_str, "%H:%M").time()
    except ValueError as e:
        print(f"Error parsing time: {time_str} - {str(e)}")
        return None


In [24]:
# Merge the datasets
# Merge on common columns: Local Time, Sex, Event, and Round
merged_df = pd.merge(
    events_df,
    results_df,
    on=["Local Time", "Sex", "Event", "Round"],
    how="inner"
)
print("\nMerged DataFrame Info:")
print(merged_df.info())


Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3498 entries, 0 to 3497
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Local Time  3498 non-null   object 
 1   Sex         3498 non-null   object 
 2   Event       3498 non-null   object 
 3   Round       3498 non-null   object 
 4   Position    3498 non-null   object 
 5   Bib Number  3498 non-null   float64
 6   Name        3498 non-null   object 
 7   Country     3498 non-null   object 
dtypes: float64(1), object(7)
memory usage: 218.8+ KB
None


In [30]:
# Filter for final rounds and group by event and sex
final_results = merged_df[merged_df['Round'] == 'Final']
medalists = final_results.groupby(['Event', 'Sex']).head(3).copy()

# Recalculate medals
def medal_count(Position):
    if Position == 1:
        return "Gold"
    elif Position == 2:
        return "Silver"
    elif Position == 3:
        return "Bronze"
    else:
        return "No Medal"

medalists['Medal'] = medalists['Position'].apply(medal_count)
country_medal_counts = medalists.groupby('Country')['Medal'].value_counts().unstack(fill_value=0)
country_medal_counts['Total'] = country_medal_counts[['Gold', 'Silver', 'Bronze']].sum(axis=1)


In [31]:
merged_df

,Local Time,Sex,Event,Round,Position,Bib Number,Name,Country,Medal
0,08:00,Men,20 Kilometres Race Walk,Final,1.0,29.0,Brian Daniel PINTADO,ECU,Gold
1,08:00,Men,20 Kilometres Race Walk,Final,2.0,15.0,Caio BONFIM,BRA,Silver
2,08:00,Men,20 Kilometres Race Walk,Final,3.0,33.0,Álvaro MARTÍN,ESP,Bronze
3,08:00,Men,20 Kilometres Race Walk,Final,4.0,55.0,Massimo STANO,ITA,No Medal
4,08:00,Men,20 Kilometres Race Walk,Final,5.0,18.0,Evan DUNFEE,CAN,No Medal
...,...,...,...,...,...,...,...,...,...
3493,08:00,Women,Marathon,Final,N/A,1917.0,Melat Yisak KEJETA,GER,No Medal
3494,08:00,Women,Marathon,Final,N/A,2374.0,Jackline SAKILU,TAN,No Medal
3495,08:00,Women,Marathon,Final,N/A,2297.0,Joan Chelimo MELLY,ROU,No Medal
3496,08:00,Women,Marathon,Final,N/A,1522.0,Sinead DIVER,AUS,No Medal


In [32]:
# Step 10: Save the merged dataset
merged_df.to_csv("combined_paris_athletics_data.csv", index=False)

print("\nMerging and cleaning complete! Combined file saved as 'combined_paris_athletics_data.csv'.")
print("\nMerged DataFrame Info:")
print(merged_df.info())


Merging and cleaning complete! Combined file saved as 'combined_paris_athletics_data.csv'.

Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3498 entries, 0 to 3497
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Local Time  3498 non-null   object 
 1   Sex         3498 non-null   object 
 2   Event       3498 non-null   object 
 3   Round       3498 non-null   object 
 4   Position    3498 non-null   object 
 5   Bib Number  3498 non-null   float64
 6   Name        3498 non-null   object 
 7   Country     3498 non-null   object 
 8   Medal       3498 non-null   object 
dtypes: float64(1), object(8)
memory usage: 246.1+ KB
None
